In [6]:
 # install PySpark
!pip install pyspark
!pip install spark-nlp
!pip install nltk
!pip install plotly
!pip install matplotlib
!pip install scikit-learn

StatementMeta(5a58fef9-6d5a-48be-a208-b669ac871209, 49, 8, Finished, Available)

     |████████████████████████████████| 540 kB 9.3 MB/s eta 0:00:01


In [3]:
%%configure -f \
{"conf": {"spark.jars.packages": "com.johnsnowlabs.nlp:spark-nlp_2.12:5.1.2"}}

StatementMeta(5a58fef9-6d5a-48be-a208-b669ac871209, 49, -1, Finished, Available)

Unrecognized options: 

In [4]:
spark

StatementMeta(5a58fef9-6d5a-48be-a208-b669ac871209, 49, 6, Finished, Available)

In [7]:
import pandas as pd
import re
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import json
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from sparknlp.annotator import *
from sparknlp.base import *
import sparknlp
from sparknlp.pretrained import PretrainedPipeline



StatementMeta(5a58fef9-6d5a-48be-a208-b669ac871209, 49, 9, Finished, Available)

In [8]:
import os
os.getcwd()

StatementMeta(5a58fef9-6d5a-48be-a208-b669ac871209, 49, 10, Finished, Available)

'/synfs/notebook/49/aml_notebook_mount'

In [11]:
dir_name = "Users/fall-2023-reddit-project-team-02/data/csv/"
df = pd.read_csv(dir_name + "nlp_reddit_df.csv")
print(df.shape)

StatementMeta(5a58fef9-6d5a-48be-a208-b669ac871209, 49, 13, Finished, Available)

(217394, 57)


In [14]:
raw_df = df.copy(deep=True)

StatementMeta(5a58fef9-6d5a-48be-a208-b669ac871209, 49, 16, Finished, Available)

## Clean Data for ML Pipeline

In [13]:
df.selftext.isna().sum()

StatementMeta(5a58fef9-6d5a-48be-a208-b669ac871209, 49, 15, Finished, Available)

82012

In [15]:
# remove rows that don't have text in selftext
df = df.dropna(subset=['selftext'])
print(df.shape)

# remove sentence embeddings which was already in the nlp df
df = df.drop('sentence_embeddings', axis = 'columns')
print(df.shape)

StatementMeta(5a58fef9-6d5a-48be-a208-b669ac871209, 49, 17, Finished, Available)

(135382, 57)
(135382, 56)


In [22]:
# extract string from sentiment analysis
df.loc[:, 'sentiment_label'] = df.loc[:, 'sentiment'].apply(lambda x: x[2:len(x)-2])
# convert boolean fields into integer fields
df.loc[:, ['airbnb', 'rent', 'gentrification', 'public_transit', 'tourists']] = df.loc[:, ['airbnb', 'rent', 'gentrification', 'public_transit', 'tourists']].astype(int)

# subset to only positive and negative since doing binary classification
sentiment_df = df[df.sentiment_label.apply(lambda x: x in ['pos', 'neg'])]
sentiment_df.loc[:,'sentiment_num'] = sentiment_df['sentiment_label'].apply(lambda x: 1 if 'pos' else 0)
sentiment_df.sentiment_label.value_counts()

StatementMeta(5a58fef9-6d5a-48be-a208-b669ac871209, 49, 24, Finished, Available)

/home/trusted-service-user/cluster-env/env/lib/python3.8/site-packages/pandas/core/indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
/home/trusted-service-user/cluster-env/env/lib/python3.8/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


pos    76152
neg    34071
Name: sentiment_label, dtype: int64

In [25]:
# create a sample of the dataframe due to size limit constraints
frac_samp = .50

# for sentiment
df_sent_sample = sentiment_df.sample(frac = frac_samp, random_state = 22)
print(df_sent_sample.shape)
spark_sent = spark.createDataFrame(df_sent_sample) 

# for subreddit
df_sub_sample = df.sample(frac = frac_samp, random_state = 22)
spark_sub = spark.createDataFrame(df_sub_sample) 
print(df_sub_sample.shape)

StatementMeta(5a58fef9-6d5a-48be-a208-b669ac871209, 49, 27, Finished, Available)

(11022, 58)
(13538, 57)


In [36]:
# from pyspark.sql import SparkSession
# from pyspark.ml import Pipeline
# from sparknlp.base import DocumentAssembler, SentenceDetector, Tokenizer, Normalizer, StopWordsCleaner
# from sparknlp.annotator import UniversalSentenceEncoder
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LinearSVC
from pyspark.ml.classification import RandomForestClassifier, DecisionTreeClassifier
from pyspark.ml import PipelineModel
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
# from pyspark.ml.classification import SVMModel, SVM
from pyspark.ml.tuning import ParamGridBuilder
from pyspark.ml.feature import OneHotEncoder, StringIndexer, IndexToString, VectorAssembler
from pyspark.ml.classification import RandomForestClassifier, MultilayerPerceptronClassifier, GBTClassifier
from pyspark.ml import Pipeline, Model

StatementMeta(5a58fef9-6d5a-48be-a208-b669ac871209, 49, 38, Finished, Available)

# Decision Tree w/ 8 Features; Predicting Subreddit

We started off initially only using 2 features in a Decision Tree in order to analyze computation time and to establish a **very** simple baseline model. 

In [101]:
# Convert the categorical labels in the 'Species' column to numerical values
label_indexer = StringIndexer(inputCol="subreddit", outputCol="label")
data = label_indexer.fit(spark_sub).transform(spark_sub)

# Assemble the feature columns into a single vector column
# assembler = VectorAssembler(inputCols=["num_words", "score"], outputCol="features")
assembler = VectorAssembler(inputCols=['num_comments', "num_words", 'score', 'airbnb', 'rent', 'gentrification', 'public_transit', 'tourists'], outputCol="features")
data = assembler.transform(data)

# Split data into training and testing sets
train_data, test_data = data.randomSplit([0.8, 0.2], seed=42)
dt_classifier = DecisionTreeClassifier(labelCol="label", featuresCol="features")

model = dt_classifier.fit(train_data)
predictions = model.transform(test_data)

evaluator_acc = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator_acc.evaluate(predictions)

print(f"Test Accuracy: {accuracy:.2f}")

evaluator_prec = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="weightedPrecision")
roc_auc = evaluator_prec.evaluate(predictions)
print(f"Test Weighted Precision: {roc_auc:.2f}")

evaluator_rec = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="weightedRecall")
roc_auc = evaluator_rec.evaluate(predictions)
print(f"Test Weighted Recall: {roc_auc:.2f}")

evaluator_f1 = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="f1")
f1_score = evaluator_f1.evaluate(predictions, {evaluator_f1.metricName: "f1"})
print(f"Test F1: {f1_score:.2f}")

StatementMeta(5a58fef9-6d5a-48be-a208-b669ac871209, 49, 103, Finished, Available)

Test Accuracy: 0.43
Test Weighted Precision: 0.41
Test Weighted Recall: 0.43
Test F1: 0.40


In [104]:
model.save("Users/fall-2023-reddit-project-team-02/data/models/rf-sub")

StatementMeta(5a58fef9-6d5a-48be-a208-b669ac871209, 49, 106, Finished, Available)

Py4JJavaError: An error occurred while calling o14508.save.
: java.io.IOException: Path Users/fall-2023-reddit-project-team-02/data/models/rf-sub already exists. To overwrite it, please use write.overwrite().save(path) for Scala and use write().overwrite().save(path) for Java and Python.
	at org.apache.spark.ml.util.FileSystemOverwrite.handleOverwrite(ReadWrite.scala:683)
	at org.apache.spark.ml.util.MLWriter.save(ReadWrite.scala:167)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.Cal

In [88]:
feature_importance = model.featureImportances.toArray()

# Show feature importance
for i, column in enumerate(assembler.getInputCols()):
    print(f"Feature '{column}': {feature_importance[i]:.2f}")

dt_sub_pred = predictions.toPandas()
dt_sub_pred.to_csv(dir_name + "pred_sub_dt.csv", index = False)

StatementMeta(5a58fef9-6d5a-48be-a208-b669ac871209, 49, 90, Finished, Available)

Feature 'num_comments': 0.60
Feature 'num_words': 0.13
Feature 'score': 0.26
Feature 'airbnb': 0.00
Feature 'rent': 0.00
Feature 'gentrification': 0.00
Feature 'public_transit': 0.00
Feature 'tourists': 0.00


/opt/spark/python/lib/pyspark.zip/pyspark/sql/pandas/conversion.py:87: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  Unsupported type in conversion to Arrow: VectorUDT
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
  warnings.warn(msg)


## Random Forest w/ Two Features; Predicting Subreddit

In [99]:
# Convert the categorical labels in the 'Species' column to numerical values
label_indexer = StringIndexer(inputCol="subreddit", outputCol="label")
data = label_indexer.fit(spark_sub).transform(spark_sub)

# Assemble the feature columns into a single vector column
assembler = VectorAssembler(inputCols=["num_words", 'score', 'airbnb', 'rent', 'gentrification', 'public_transit', 'tourists'], outputCol="features")
data = assembler.transform(data)

# Split data into training and testing sets
train_data, test_data = data.randomSplit([0.8, 0.2], seed=42)
rf = RandomForestClassifier(labelCol="label", featuresCol="features", numTrees=20)

model = rf.fit(train_data)
predictions = model.transform(test_data)

evaluator_acc = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator_acc.evaluate(predictions)

print(f"Test Accuracy: {accuracy:.2f}")

evaluator_prec = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="weightedPrecision")
roc_auc = evaluator_prec.evaluate(predictions)
print(f"Test Weighted Precision: {roc_auc:.2f}")

evaluator_rec = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="weightedRecall")
roc_auc = evaluator_rec.evaluate(predictions)
print(f"Test Weighted Recall: {roc_auc:.2f}")

evaluator_f1 = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="f1")
f1_score = evaluator_f1.evaluate(predictions, {evaluator_f1.metricName: "f1"})
print(f"Test F1: {f1_score:.2f}")

StatementMeta(5a58fef9-6d5a-48be-a208-b669ac871209, 49, 101, Finished, Available)

Test Accuracy: 0.40
Test Weighted Precision: 0.39
Test Weighted Recall: 0.40
Test F1: 0.36


In [100]:
feature_importance = model.featureImportances.toArray()

# Show feature importance
for i, column in enumerate(assembler.getInputCols()):
    print(f"Feature '{column}': {feature_importance[i]:.2f}")

sub_pred = predictions.toPandas()
sub_pred.to_csv(dir_name + "pred_sub_rf.csv", index = False)

StatementMeta(5a58fef9-6d5a-48be-a208-b669ac871209, 49, 102, Finished, Available)

Feature 'num_words': 0.32
Feature 'score': 0.63
Feature 'airbnb': 0.00
Feature 'rent': 0.03
Feature 'gentrification': 0.00
Feature 'public_transit': 0.02
Feature 'tourists': 0.00


## SVM; predicting sentiment

In [91]:
# Convert the categorical labels in the 'sentiment label' column to numerical values
label_indexer = StringIndexer(inputCol="sentiment_label", outputCol="label")
data = label_indexer.fit(spark_sent).transform(spark_sent)

# Assemble the feature columns into a single vector column
assembler = VectorAssembler(inputCols=["num_words", 'score', 'airbnb', 'rent', 'gentrification', 'public_transit', 'tourists'], outputCol="features")
data = assembler.transform(data)

# Split data into training and testing sets
train_data, test_data = data.randomSplit([0.8, 0.2], seed=42)
svm_classifier = LinearSVC(maxIter=3, regParam=0.1, featuresCol="features", labelCol="label")

print("Starting to fit")
model = svm_classifier.fit(train_data)
print("finished fitting model")
predictions = model.transform(test_data)


evaluator_acc = BinaryClassificationEvaluator(labelCol="label", rawPredictionCol="prediction", metricName="areaUnderROC")
accuracy = evaluator_acc.evaluate(predictions)

print(f"Test areaUnderROC: {accuracy:.2f}")

evaluator_prec = BinaryClassificationEvaluator(labelCol="label", rawPredictionCol="prediction", metricName="areaUnderPR")
roc_auc = evaluator_prec.evaluate(predictions)
print(f"Test areaUnderPR: {roc_auc:.2f}")

evaluator_acc = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator_acc.evaluate(predictions)

print(f"Test Accuracy: {accuracy:.2f}")

evaluator_prec = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="weightedPrecision")
roc_auc = evaluator_prec.evaluate(predictions)
print(f"Test Weighted Precision: {roc_auc:.2f}")

evaluator_rec = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="weightedRecall")
roc_auc = evaluator_rec.evaluate(predictions)
print(f"Test Weighted Recall: {roc_auc:.2f}")

evaluator_f1 = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="f1")
f1_score = evaluator_f1.evaluate(predictions, {evaluator_f1.metricName: "f1"})
print(f"Test F1: {f1_score:.2f}")

StatementMeta(5a58fef9-6d5a-48be-a208-b669ac871209, 49, 93, Finished, Available)

Starting to fit
finished fitting model
Test areaUnderROC: 0.51
Test areaUnderPR: 0.41
Test Accuracy: 0.68
Test Weighted Precision: 0.62
Test Weighted Recall: 0.68
Test F1: 0.57


In this example, coefficients will contain the weights associated with each feature. Keep in mind that the interpretation of the weights can be more complex for SVM models, and it may not provide a direct measure of feature importance as in other algorithms like decision trees.

You can interpret the weights in relation to the magnitude and sign. Larger magnitude weights indicate a stronger influence on the decision boundary, and the sign of the weight can indicate the direction of influence.

If feature scaling was applied before training the model, you should consider the scaled values when interpreting the coefficients. Additionally, you may need to consider the absolute values of the coefficients for magnitude-based importance interpretation.

In [92]:
# Create a mapping between feature names and their corresponding coefficients
feature_coefficients = dict(zip(assembler.getInputCols(),
                            model.coefficients))


# Print each coefficient next to its corresponding feature
for feature_name, coefficient in feature_coefficients.items():
    print(f"Feature: {feature_name}, Coefficient: {coefficient}")

sub_pred = predictions.toPandas()
sub_pred.to_csv(dir_name + "pred_sent_svm.csv", index = False)

StatementMeta(5a58fef9-6d5a-48be-a208-b669ac871209, 49, 94, Finished, Available)

Feature: num_words, Coefficient: 0.000868626019672739
Feature: score, Coefficient: 0.0013619016699627539
Feature: airbnb, Coefficient: 0.007097757497507614
Feature: rent, Coefficient: 0.5685826071918254
Feature: gentrification, Coefficient: 0.8830408662448551
Feature: public_transit, Coefficient: 1.1295979523817465
Feature: tourists, Coefficient: 0.5925693525651131


## Random Forest; Predicting Sentiment

In [111]:
# Convert the categorical labels in the 'Species' column to numerical values
label_indexer = StringIndexer(inputCol="sentiment_label", outputCol="label")
data = label_indexer.fit(spark_sent).transform(spark_sent)

# Assemble the feature columns into a single vector column
assembler = VectorAssembler(inputCols=["num_words", 'score', 'airbnb', 'rent', 'gentrification', 'public_transit', 'tourists'], outputCol="features")
data = assembler.transform(data)

# Split data into training and testing sets
train_data, test_data = data.randomSplit([0.8, 0.2], seed=42)
rf_classifier = RandomForestClassifier(labelCol="label", featuresCol="features", numTrees=50, seed=42)

print("Starting to fit")
model = rf_classifier.fit(train_data)
print("finished fitting model")
predictions = model.transform(test_data)

# evaluate
evaluator_acc = BinaryClassificationEvaluator(labelCol="label", rawPredictionCol="prediction", metricName="areaUnderROC")
accuracy = evaluator_acc.evaluate(predictions)

print(f"Test areaUnderROC: {accuracy:.2f}")

evaluator_prec = BinaryClassificationEvaluator(labelCol="label", rawPredictionCol="prediction", metricName="areaUnderPR")
roc_auc = evaluator_prec.evaluate(predictions)
print(f"Test areaUnderPR: {roc_auc:.2f}")

evaluator_acc = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator_acc.evaluate(predictions)

print(f"Test Accuracy: {accuracy:.2f}")

evaluator_prec = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="weightedPrecision")
roc_auc = evaluator_prec.evaluate(predictions)
print(f"Test Weighted Precision: {roc_auc:.2f}")

evaluator_rec = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="weightedRecall")
roc_auc = evaluator_rec.evaluate(predictions)
print(f"Test Weighted Recall: {roc_auc:.2f}")

evaluator_f1 = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="f1")
f1_score = evaluator_f1.evaluate(predictions, {evaluator_f1.metricName: "f1"})
print(f"Test F1: {f1_score:.2f}")

StatementMeta(5a58fef9-6d5a-48be-a208-b669ac871209, 49, 113, Finished, Available)

Starting to fit
finished fitting model
Test areaUnderROC: 0.52
Test areaUnderPR: 0.46
Test Accuracy: 0.68
Test Weighted Precision: 0.65
Test Weighted Recall: 0.68
Test F1: 0.58


In [112]:
model.save("Users/fall-2023-reddit-project-team-02/data/models/rf-sent")

StatementMeta(5a58fef9-6d5a-48be-a208-b669ac871209, 49, 114, Submitted, Running)

In [98]:
feature_importance = model.featureImportances.toArray()

# Show feature importance
for i, column in enumerate(assembler.getInputCols()):
    print(f"Feature '{column}': {feature_importance[i]:.2f}")

sub_pred = predictions.toPandas()
sub_pred.to_csv(dir_name + "pred_sent_rf.csv", index = False)

StatementMeta(5a58fef9-6d5a-48be-a208-b669ac871209, 49, 100, Finished, Available)

Feature 'num_words': 0.43
Feature 'score': 0.46
Feature 'airbnb': 0.00
Feature 'rent': 0.02
Feature 'gentrification': 0.00
Feature 'public_transit': 0.07
Feature 'tourists': 0.01


/opt/spark/python/lib/pyspark.zip/pyspark/sql/pandas/conversion.py:87: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  Unsupported type in conversion to Arrow: VectorUDT
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
  warnings.warn(msg)


## Use Sentence Embeddings for Subreddit Predictions

In [109]:
# Step 1: Transforms raw title into 'document' annotation
documentAssembler = DocumentAssembler()\
    .setInputCol("selftext")\
    .setOutputCol("document")

# Step 2: Annotate sentences in the document
sentenceDetector = SentenceDetector()\
    .setInputCols(["document"])\
    .setOutputCol("sentence")

# Step 3: Tokenize the sentences into tokens
tokenizer = Tokenizer()\
    .setInputCols(["sentence"])\
    .setOutputCol("token")

# Step 4: Normalize the tokens
normalizer = Normalizer()\
    .setInputCols(["token"])\
    .setOutputCol("normalized")\
    .setLowercase(True)\
    .setCleanupPatterns(["[^\w\d\s]"])  # Removes any non-word, non-digit, non-whitespace characters

# Step 5: Clean tokens using the StopWordsCleaner
stop_words_cleaner = StopWordsCleaner().pretrained("stopwords_iso", "en")\
    .setInputCols(["normalized"])\
    .setOutputCol("cleanTokens")\
    .setCaseSensitive(False)

# Step 6: Load a pretrained Universal Sentence Encoder model
use = UniversalSentenceEncoder.pretrained(name="tfhub_use", lang="en")\
    .setInputCols(["document"])\
    .setOutputCol("sentence_embeddings")

# Step 7: Load a pretrained SentimentDL model for sentiment analysis
# sentimentdl = SentimentDLModel.pretrained(lang="en")\
#     .setInputCols(["sentence_embeddings"])\
#     .setOutputCol("sentiment")
assembler = VectorAssembler(inputCols=["sentence_embeddings"], outputCol="features")

rf_classifier = RandomForestClassifier(labelCol="subreddit", featuresCol="features", numTrees=10)


# Construct the ML pipeline
pipeline = Pipeline(
    stages=[
        documentAssembler,
        sentenceDetector,
        tokenizer,
        normalizer,
        stop_words_cleaner,
        use,
        assembler,
        rf_classifier
    ])


StatementMeta(5a58fef9-6d5a-48be-a208-b669ac871209, 49, 111, Finished, Available)

stopwords_iso download started this may take some time.
Approximate size to download 2.1 KB
[OK!]
tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[OK!]


In [ ]:
# Split Data and Train the Model
(train_data, test_data) = df_sub_sample.randomSplit([0.8, 0.2], seed=42)

# Train the pipeline
model = pipeline.fit(train_data)

# Make Predictions and Evaluate
predictions = model.transform(test_data)

from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
